In [ ]:
from PIL import Image

In [ ]:
def text_to_binary(text):
    """Converts a string of text to its binary representation."""
    binary_string = ''.join(format(ord(char), '08b') for char in text)
    return binary_string

def binary_to_text(binary):
    """Converts a binary string back to text."""
    text = ''.join(chr(int(binary[i:i+8], 2)) for i in range(0, len(binary), 8))
    return text

def encode_image(image_path, secret_message, output_path):
    """Encodes a secret message into an image using LSB steganography."""
    try:
        img = Image.open(image_path).convert("RGB")
        width, height = img.size
        binary_message = text_to_binary(secret_message + "***END***")  # Add delimiter
        message_length = len(binary_message)
        pixel_index = 0
        bit_index = 0
        if message_length > width * height * 3:
            raise ValueError("Message is too long to encode in this image.")

        for x in range(width):
            for y in range(height):
                pixel = list(img.getpixel((x, y)))
                for i in range(3):  # Iterate through R, G, B components
                    if bit_index < message_length:
                        # Clear the least significant bit and set it to the message bit
                        pixel[i] = (pixel[i] & ~1) | int(binary_message[bit_index])
                        bit_index += 1
                    else:
                        break  # Message fully encoded
                img.putpixel((x, y), tuple(pixel))
                if bit_index >= message_length:
                    break
            if bit_index >= message_length:
                break

        img.save(output_path)
        print(f"Message successfully encoded and saved to {output_path}")

    except FileNotFoundError:
        print("Error: Image file not found.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [3]:
def decode_image(image_path):
    """Decodes a secret message from an image using LSB steganography."""
    try:
        img = Image.open(image_path).convert("RGB")
        width, height = img.size
        binary_message = ""
        decoded_message = ""

        for x in range(width):
            for y in range(height):
                pixel = list(img.getpixel((x, y)))
                for i in range(3):
                    binary_message += str(pixel[i] & 1)
                    if binary_message.endswith("001010100010101000101010"): # "***END***" in binary
                        decoded_message = binary_to_text(binary_message[:-24])
                        print("Message decoded successfully!")
                        return decoded_message
        print("No hidden message found or incorrect encoding.")
        return None

    except FileNotFoundError:
        print("Error: Image file not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

if __name__ == "__main__":
    while True:
        print("\nSteganography Tool")
        print("1. Encode a message into an image")
        print("2. Decode a message from an image")
        print("3. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            image_path = input("Enter the path to the image file: ")
            secret_message = input("Enter the secret message to encode: ")
            output_path = input("Enter the desired output path for the encoded image: ")
            encode_image(image_path, secret_message, output_path)
        elif choice == '2':
            image_path = input("Enter the path to the image file to decode: ")
            decoded_message = decode_image(image_path)
            if decoded_message:
                print("Decoded message:", decoded_message)
        elif choice == '3':
            print("Exiting the tool.")
            break
        else:
            print("Invalid choice. Please try again.")

KeyboardInterrupt: Interrupted by user